# Graph erstellen

In [69]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

def get_centroid(polygons):
    '''
    Fügt den Zentroid zu den Polygonen hinzu
    polygons: geladene Polygon-Shapefile-Daten
    '''
    polygons['centroid'] = polygons['geometry'].centroid
    return polygons

def get_closest_point(line, point):
    '''Ermittelt den nächstgelegenen Punkt auf der Linie zu einem Punkt'''
    closest_point = line.interpolate(line.project(point))
    return closest_point

def closest_points_buildings(polygons, lines):
    '''Ermittelt für jedes Polygon mit WB_HU > 0 den nächsten Punkt (Anschlusspunkt) auf dem Linien-Netzwerk und die street-ID'''
    list=[]
    # Iteration über jeden Polygon-Zentroiden
    for index, row_p in polygons.iterrows():
        if row_p['WB_HU'] > 0:
            centroid = row_p['centroid']

            # WLD an der das Gebäude angeschlossen ist
            wld_id = row_p['WLD_ID']
            
            connected_line = lines[lines['WLD_ID']==wld_id]
            try:
                index_val = connected_line.index[0]
            except:
                list.append(index)

            # Ermitteln Sie den nächsten Punkt auf dieser Linie
            closest_point = get_closest_point(lines.at[index_val, 'geometry'], centroid)
            

            polygons.at[index, 'Anschlusspunkt'] = closest_point
            #polygons.at[index, 'wld_index'] = int(connected_line)
    
    return polygons,list

def closest_points_sources(sources, lines):
    '''Ermittelt für jede Quelle den nächsten Punkt (Anschlusspunkt) auf dem Linien-Netzwerk und die street-ID'''
    # Iteration über jede Quelle und Suche des nächstgelegenen Punkts auf dem Linien-Netzwerk
    for index, row_s in sources.iterrows():

        # Initialisieren der Variablen für den minimalen Abstand und den nächstgelegenen Punkt
        min_distance = float('inf')
        closest_point = None
        source = row_s['geometry']

        # Iteration über jede Linie im Linien-Netzwerk
        for idx,row in lines.iterrows():
            line_coords = list(row['geometry'].coords)  # Liste der Punkte, aus denen die Linie besteht
            
            # Iteration über jeden Linienabschnitt, um den nächstgelegenen Punkt zu finden
            for i in range(1, len(line_coords)):
                start_point = Point(line_coords[i-1])
                end_point = Point(line_coords[i])
                line_segment = LineString([start_point, end_point])

                distance = line_segment.distance(source)

                if distance < min_distance:
                    min_distance = distance
                    closest_point = get_closest_point(LineString([start_point, end_point]), source)
                    id = idx
        sources.at[index, 'Anschlusspunkt'] = closest_point
        sources.at[index, 'wld_index'] = int(id)
    return sources

def add_connection_to_streets(buildings, streets):
    '''Anschlusspunkte in die Straßenlinien einfügen'''

    for df in [buildings]:
        for index, row in df.iterrows():
            wld_id = row['WLD_ID']
            wld_index = wld[wld['WLD_ID']==wld_id].index
            if not pd.isna(wld_id):
                anschlusspunkt = row['Anschlusspunkt']
                line = streets['geometry'][wld_index]
                print(line)
                line_coords = list(line.coords)

                insertion_position = None
                min_distance = float('inf')

                # Einfügeposition in der Linie finden
                for i in range(1, len(line_coords)):
                    segment = LineString([line_coords[i-1], line_coords[i]])
                    distance = segment.distance(anschlusspunkt)

                    if distance < min_distance:
                        min_distance = distance
                        insertion_position = i

                # Fügen Sie den ursprünglichen Anschlusspunkt in die Linienkoordinaten ein
                if (anschlusspunkt.x, anschlusspunkt.y) not in line_coords:
                    line_coords.insert(insertion_position, (anschlusspunkt.x, anschlusspunkt.y))
                    streets.at[wld_index, 'geometry'] = LineString(line_coords)
    return streets

def create_street_network(streets):
    '''Graph (Straßen-Netzwerk) erstellen, Knoten und Kannten hinzufügen'''
    # Graph erstellen
    G = nx.Graph()

    # Knoten hinzufügen
    for idx, row in streets.iterrows():
        geom = row['geometry']
        line_coords = list(geom.coords)

        # Iteration über jeden Punkt auf der Linie
        for i in range(len(line_coords)):
            node = line_coords[i]
            G.add_node(node)

            # Verbindung zu vorherigem Punkt (außer beim ersten Punkt)
            if i > 0:
                prev_node = line_coords[i-1]
                G.add_edge(node, prev_node)
    return G

def connect_centroids(G, buildings):
    '''Fügt die Zentroiden der Gebäude zum Netz G hinzu'''
    for index, row in buildings.iterrows():
        centroid = row['centroid']
        closest_point = row['Anschlusspunkt']
        if not pd.isna(closest_point):
            G.add_edge(centroid.coords[0], (closest_point.x, closest_point.y))
    return G

def connect_source(G, sources):
    '''Fügt Energiequelle dem Netz G hinzu'''
    for index, row in sources.iterrows():
        source = row['geometry']
        closest_point = row['Anschlusspunkt']
        if not pd.isna(source):
            G.add_edge(source.coords[0], (closest_point.x, closest_point.y))
    return G

def add_attribute_length(G):
    # Kantenattribute hinzufügen
    for node1, node2 in G.edges():
        geom = LineString([node1, node2])
        G.edges[node1, node2]['length'] = geom.length
    return G

def graph(buildings, streets):
    '''Baut das Netz(Graph) aus Gebäuden, Straßen und Quellen'''

    # Hinzufügen der Zentroide zu den Polygonen
    buildings = get_centroid(buildings)

    # Berechnung der nächstgelegenen Punkte für Gebäude und Quellen
    buildings,list = closest_points_buildings(buildings, streets)
    #sources = closest_points_sources(sources, streets)

    # Anschlusspunkte in die Straßenlinien einfügen
    streets = add_connection_to_streets(buildings, streets)
    
    # Graph (Straßennetz) erstellen 
    G = create_street_network(streets)

    # Zentroiden der Gebäude an Graph anschließen
    G = connect_centroids(G, buildings)

    # Quellen an Graph anschließen
    #G = connect_source(G,sources) 

    # Attribute hinzufügen
    G = add_attribute_length(G)

    return G,list

def plot_G(G):

    # Koordinatensystem festlegen
    pos = {node: (node[0], node[1]) for node in G.nodes}

    # Graph anzeigen
    plt.figure(figsize=(50, 50))
    nx.draw_networkx(G, pos=pos, with_labels=False, font_size=6, node_size=3, node_color='blue', edge_color='gray')
    plt.show()

# Laden der Shapefiles
polygon_file = r'C:\Users\Lars_Goray\OneDrive - FH Muenster\Dokumente\GitHub\mshack23-waermeplanung\data\buildings.parquet'
#source_file = "test_source.shp"

buildings = gpd.read_parquet(polygon_file)
buildings = buildings.to_crs(25832)
#sources = gpd.read_file(source_file)

wld_file = r'C:\Users\Lars_Goray\OneDrive - FH Muenster\Dokumente\GitHub\mshack23-waermeplanung\data\heat_lines.parquet'
wld = gpd.read_parquet(wld_file)
wld = wld.to_crs(25832)

G,list = graph(buildings,wld)
plot_G(G)

GeoSeries([], Name: geometry, dtype: geometry)


AttributeError: 'GeoSeries' object has no attribute 'coords'

In [ ]:
buildings = buildings.drop(list)

# Netz-Analyse

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

def shortest_path(G, start_point, end_point, weight='length'):
    '''
    Berechnet den Kürzesten Pfad von start_point zu end_point auf G mit dem Attribut weight
    '''
    # Kürzesten Pfad berechnen
    path = nx.shortest_path(G, start_point, end_point, weight=weight)
    
    # Ausgabe des Ergebnisses
    return path

def network_analysis(G, buildings, sources, weight='length'):
    '''Berechnet das Netz, indem der kürzeste Pfad zu jedem Gebäude gesucht wird'''

    buildings = buildings[buildings['HU_calc'] > 0]
    start_point = (sources['geometry'][0].x, sources['geometry'][0].y)7.63808 51.95156
    
    # Neuer Graph für das Netzwerk erstellen
    net = nx.Graph()

    for row in buildings.itertuples():
        end_point = (row.centroid.x, row.centroid.y)
        
        # Die Leistung des Gebäudes extrahieren
        power = row.VOL
        
        path = shortest_path(G, start_point, end_point, weight)
        
        # Hinzufügen der Knoten und Kanten des Pfades zum Netzwerkgraphen
        for u, v in zip(path[:-1], path[1:]):
            # Kopiere alle Kantenattribute
            net.add_edge(u, v, **G.edges[u, v])
            # Aktualisiere die Leistung
            net.edges[u, v]['power'] = net.edges[u, v].get('power', 0) + power
    return net

def plot_network(G, net, buildings, sources):
    '''
    Zeigt das Straßennetzwerk G und das berechnete Netz net an.
    G: Straßennetzwerk-Graph
    net: Berechnetes Netz
    buildings: GeoDataFrame der Gebäude
    sources: GeoDataFrame der Energiequellen
    '''
    # Positionen der Knoten
    pos = {node: (node[0], node[1]) for node in G.nodes}

    # Figure und Axes erstellen
    fig, ax = plt.subplots(figsize=(20, 20))

    # Straßen zeichnen
    nx.draw_networkx_edges(G, pos=pos, ax=ax, edge_color='gray', alpha=0.7)

    # Kürzeste Pfade zeichnen
    for path in net.edges:
        nx.draw_networkx_edges(net, pos=pos, edgelist=[path], ax=ax, edge_color='blue', width=1.0)

    # Gebäude zeichnen
    buildings.plot(ax=ax, facecolor='#ff8888', edgecolor='black')

    # Energiequelle als Punkt zeichnen
    sources.plot(ax=ax, marker='o', markersize=15, color='green')

    # Raster und Achsentitel aktivieren
    #ax.grid(True)
    ax.set_title('Straßennetzwerk und berechnetes Netz')

    # Plot anzeigen
    plt.show()

net=network_analysis(G,buildings,sources)
#net2,i,cg=network_analysis2(G, net, buildings, sources)

plot_network(G,net,buildings,sources)
#plot_network(G,net2,buildings,sources)

# Graph zu shape

In [ ]:
from shapely.geometry import LineString

def ensure_power_attribute(graph):
    """
    Stellt sicher, dass jede Kante im Graphen das Attribut 'power' hat.
    Wenn eine Kante das Attribut nicht hat, wird es mit dem Wert 0 initialisiert.

    Parameters:
    - graph: Ein NetworkX-Graph

    Returns:
    - Der modifizierte Graph mit dem Attribut 'power' für jede Kante.
    """
    for u, v in graph.edges():
        if 'power' not in graph[u][v]:
            graph[u][v]['power'] = 0
    return graph

def graph_to_gdf(G):
    """Konvertiert einen networkx-Graphen in ein GeoDataFrame, wobei auch die Kantenattribute übernommen werden."""
    # Sammeln Sie die Kanten des Graphen und konvertieren Sie sie in LineString-Objekte
    # und sammeln Sie auch die Kantenattribute
    geometries = []
    attributes = {}

    for u, v, data in G.edges(data=True):
        geometries.append(LineString([u, v]))

        # Sammeln Sie Attribute für jede Kante
        for key, value in data.items():
            if key in attributes:
                attributes[key].append(value)
            else:
                attributes[key] = [value]

    # Erstellen Sie ein GeoDataFrame aus den LineString-Objekten und den Attributen
    gdf = gpd.GeoDataFrame(attributes, geometry=geometries)

    return gdf

g = ensure_power_attribute(net)
g = graph_to_gdf(g)

# GeoDataFrame als Shapefile
name = 'Netz'
g.to_file(f"{name}.shp")